# Imports

In [32]:
import os

import pickle
import numpy as np
np.set_printoptions(precision=4)

os.environ['CUDA_VISIBLE_DEVICES'] = '' # run tensorflow without GPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
print(f'Tensorflow version {tf.__version__}')
from tensorflow import keras;
from tensorflow.keras.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# DNN Model
Multi-output regression involves predicting two or more numerical variables.

In [4]:
def create_dnn_model(n_features, 
                     layers, 
                     n_outputs=1, 
                     activation='relu', 
                     init='he_uniform', 
                     batch_normalization=False, 
                     dropout=0, 
                     k_reg=False, 
                     k_reg_lr=0.001, 
                     a_reg=False, 
                     a_reg_lr=0.001, 
                     loss='mse', 
                     optimizer='adam'):

    model = Sequential()
    
    # ============
    # input-layer
    # ============
    model.add(Dense(units=layers[0]
                      , input_dim=n_features
                      , kernel_initializer=init
                      , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                      , activity_regularizer=l2(a_reg_lr) if a_reg else None
                      , use_bias=(not batch_normalization)
                    ))
    
    
    if batch_normalization:
        model.add(BatchNormalization())
    
    model.add(Activation(activation))

    if dropout >= 0:
        model.add(Dropout(dropout))

    # ==============
    # hidden-layers
    # ==============
    if len(layers) > 1:
        for units in layers[1:]:
            model.add(Dense(units=units
                            , kernel_initializer=init
                            , kernel_regularizer=l2(k_reg_lr) if k_reg else None
                            , activity_regularizer=l2(a_reg_lr) if a_reg else None
                            , use_bias=(not batch_normalization)
                            ))

            if batch_normalization:
                model.add(BatchNormalization())

            model.add(Activation(activation))

            if dropout > 0:
                model.add(Dropout(dropout))

    # =============
    # output-layer
    # =============
    model.add(Dense(n_outputs))

    model.compile(loss=loss, metrics=[loss], optimizer=optimizer)

    return model

# Train model

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split

def train_model(model, X_train, y_train, batch_size=32):

    # split train/val
    X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                      y_train, 
                                                      test_size=0.25)

    # early-stopping
    es_patience = 50
    es = EarlyStopping(monitor='val_loss', 
                        patience=es_patience, 
                        mode='min', 
                        restore_best_weights=True, 
                        verbose=0)
    
    # reduce learning-rate on plateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=10)

    # fit model
    history = model.fit(X_train, 
                        y_train, 
                        validation_data=(X_val, y_val), 
                        epochs=200, 
                        batch_size=batch_size, 
                        shuffle=False, 
                        callbacks=[es, reduce_lr], 
                        verbose=0)

    return model, history.history

# Plot learning curves

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_learning_curves(history, train_key='loss', val_key='val_loss'):
    plt.figure(figsize=(2,2))
    plt.plot(history[train_key])
    plt.plot(history[val_key])
    plt.title('learning curves')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.show()

# Fit

In [ ]:
def fit_dnn_model(X_train, y_train):
    """
    Fit a DNN model to training data
    """
    
    # create model
    dnn_model = create_dnn_model(X_train.shape[1], 
#                             layers=[256,256], 
                            layers=[64,64], 
                            n_outputs=y_train.shape[1], 
                            activation='relu', 
                            init=keras.initializers.HeUniform(), 
                            batch_normalization=True, 
                            dropout=0.0, 
                            optimizer=Adam(learning_rate=1e-2), 
                            k_reg=True,
                            k_reg_lr=1e-5, 
                            a_reg=True,
                            a_reg_lr=1e-5)
    # train model
    dnn_model, history = train_model(dnn_model, X_train, y_train)
    return dnn_model, history

# Predict

In [34]:
def predict(model, X_test, y_test):
    
    # predict
    y_pred = model.predict_step(X_test)
    
    # mse
    mse = mean_squared_error(y_test, y_pred)

    return y_pred, float(mse)